In [21]:
import random
from abc import ABC, abstractmethod

# Classe para representar uma rainha
class Rainha:
    def __init__(self, linha, coluna):
        self.linha = linha
        self.coluna = coluna

    def __eq__(self, other):
        """Sobrescreve o método de igualdade para comparar rainhas."""
        return self.linha == other.linha and self.coluna == other.coluna

    def __str__(self):
        return f"Rainha(linha={self.linha}, coluna={self.coluna})"

# Classe para representar o tabuleiro
class Tabuleiro:
    def __init__(self, linhas, colunas):
        self.rainhas = []
        self.linhas = linhas
        self.colunas = colunas

        # self.start()

    def start(self):
        for coluna in range(self.colunas):
            linha = random.randint(0, self.linhas - 1)
            self.adicionar_rainha(Rainha(linha, coluna))

    def adicionar_rainha(self, rainha):
        self.rainhas.append(rainha)

    def remover_rainha(self, rainha):
        self.rainhas.remove(rainha)

    def calcular_conflitos(self):
        conflitos = 0
        for i in range(self.linhas):
            for j in range(i + 1, self.colunas):
                r1, r2 = self.rainhas[i], self.rainhas[j]
                if r1.linha == r2.linha or abs(r1.linha - r2.linha) == abs(r1.coluna - r2.coluna):
                    conflitos += 1
        return conflitos

    def clonar(self):
        novo_tabuleiro = Tabuleiro(self.linhas, self.colunas)
        novo_tabuleiro.rainhas = [Rainha(r.linha, r.coluna) for r in self.rainhas]
        return novo_tabuleiro

    def __str__(self):
        return "\n".join(str(rainha) for rainha in self.rainhas)

# Interface para algoritmos de resolução
class AlgoritmoResolucao(ABC):
    @abstractmethod
    def resolver(self, tabuleiro):
        pass

# Implementação do algoritmo de subida de encosta
class SubidaEncosta(AlgoritmoResolucao):
    def resolver(self, tabuleiro):
        conflitos_atual = tabuleiro.calcular_conflitos()
        
        while conflitos_atual > 0:
            melhor_tabuleiro = tabuleiro.clonar()
            melhor_conflitos = conflitos_atual
            
            for coluna in range(tabuleiro.colunas):
                rainha_atual = next((r for r in tabuleiro.rainhas if r.coluna == coluna), None)
                if not rainha_atual:
                    continue  # Não há rainha nesta coluna
                
                for linha in range(tabuleiro.linhas):
                    if linha == rainha_atual.linha:
                        continue  # Ignora a posição atual da rainha
                    
                    # Cria um novo tabuleiro com a rainha movida para a nova posição
                    novo_tabuleiro = tabuleiro.clonar()
                    novo_tabuleiro.remover_rainha(rainha_atual)
                    novo_tabuleiro.adicionar_rainha(Rainha(linha, coluna))
                    
                    novos_conflitos = novo_tabuleiro.calcular_conflitos()
                    
                    if novos_conflitos < melhor_conflitos:
                        melhor_tabuleiro = novo_tabuleiro
                        melhor_conflitos = novos_conflitos
            
            if melhor_conflitos >= conflitos_atual:
                # Estagnação, reinicia com um novo tabuleiro aleatório
                tabuleiro = self._gerar_tabuleiro_aleatorio(tabuleiro.linhas, tabuleiro.colunas)
                conflitos_atual = tabuleiro.calcular_conflitos()
            else:
                tabuleiro = melhor_tabuleiro
                conflitos_atual = melhor_conflitos
        
        return tabuleiro

    def _gerar_tabuleiro_aleatorio(self, linhas, colunas):
        tabuleiro = Tabuleiro(linhas, colunas)
        for coluna in range(colunas):
            linha = random.randint(0, linhas - 1)
            tabuleiro.adicionar_rainha(Rainha(linha, coluna))
        return tabuleiro

# Classe para visualizar o tabuleiro
class VisualizadorTabuleiro:
    @staticmethod
    def mostrar(tabuleiro):
        print("Conflitos: ", tabuleiro.calcular_conflitos())

        print("Solução encontrada:")
        for linha in range(tabuleiro.linhas):
            print(" ".join("Q" if any(r.linha == linha and r.coluna == coluna for r in tabuleiro.rainhas) else "." for coluna in range(tabuleiro.colunas)))



In [23]:
# Inicializa o tabuleiro com 8 rainhas em posições aleatórias
tabuleiro = Tabuleiro(8, 8)
tabuleiro.start()

algoritmo = SubidaEncosta()
solucao = algoritmo.resolver(tabuleiro)

VisualizadorTabuleiro.mostrar(solucao)

Conflitos:  0
Solução encontrada:
. . . Q . . . .
. . . . . . Q .
. . Q . . . . .
. . . . . . . Q
. Q . . . . . .
. . . . Q . . .
Q . . . . . . .
. . . . . Q . .


In [26]:
# Inicializa o tabuleiro com 15 rainhas em posições aleatórias
tabuleiro = Tabuleiro(15, 15)
tabuleiro.start()

algoritmo = SubidaEncosta()
solucao = algoritmo.resolver(tabuleiro)

VisualizadorTabuleiro.mostrar(solucao)

Conflitos:  0
Solução encontrada:
. . . . . Q . . . . . . . . .
. . . . . . . . . Q . . . . .
. . . . . . . . . . . Q . . .
. . . . . . . . . . . . . Q .
. . Q . . . . . . . . . . . .
. . . . . . . Q . . . . . . .
. . . . . . . . . . . . Q . .
. Q . . . . . . . . . . . . .
. . . Q . . . . . . . . . . .
. . . . . . Q . . . . . . . .
. . . . . . . . . . . . . . Q
. . . . . . . . . . Q . . . .
. . . . . . . . Q . . . . . .
. . . . Q . . . . . . . . . .
Q . . . . . . . . . . . . . .
